In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
import numpy as np
import pandas as pd

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

In [13]:
data = pd.read_csv('transactions_last_2.csv', sep=';')
data

,purchdate,amount,mcc,mrchcity,mrchname,cnum
0,2019-11-02 00:00:00,9000000.00,5691,MOSCOW,IA7D1H Z1D1T7,MJNLMQ
1,2019-11-02 00:00:00,9000000.00,5691,MOSCOW,IA7D1H Z1D1T7,MJNLMQ
2,2019-10-25 00:00:00,7716900.00,5691,MOSCOW,HXL4K TL4EEXD,ELETCO
3,2019-11-29 00:00:00,5100000.00,6513,SANKT-PETERBU,7X 1E7HXD H1DKW1IKCL,EL0KSF
4,2019-12-03 00:00:00,5023956.60,5944,MOSKVA,73 7HJXK7 (W7X),MPKIJJ
...,...,...,...,...,...,...
104161403,2019-10-08 00:00:00,0.01,5411,SAINT-PETERSB,7LIA7D ZLHH4T1J KW0,8ELH0M
104161404,2019-12-03 00:00:00,0.01,4812,CHELYABINSK,XYKI JXKE1H13XC,EEPQ69
104161405,2019-11-29 00:00:00,0.01,5812,MOSCOW,J1KEXJ7D J4I1,8EE4HF
104161406,2019-10-31 00:00:00,0.01,5814,MOSCOW,21H1D7O7 EXIA37,EE0H46


### Распределение mcc на датасете

In [8]:
print(data.shape)
data.mcc.value_counts()

(104161408, 6)


5411    27053381
5814    13208780
6011     6563269
5499     5821330
6012     5086225
          ...   
3652           1
3542           1
3544           1
3719           1
3523           1
Name: mcc, Length: 394, dtype: int64

### Предобработка + объединение датасетов

In [20]:
stores = pd.read_csv('stores.csv')
stores = stores.rename(columns={"merchant_name": "mrchname"})

df = pd.DataFrame(data.loc[data['mrchcity'] == 'MOSCOW'])
df = df.loc[df['mcc'] == 5411]
df.mcc = df.mcc.astype('object')
df = df.reset_index(drop=True)

merge_data = pd.merge(df, stores, on='mrchname', how='inner')
merge_data = merge_data.sort_values(by='purchdate', ascending=True)
merge_data = merge_data.reset_index(drop=True)
merge_data = merge_data.drop(['Unnamed: 0'], axis=1)

merge_data

,purchdate,amount,mcc,mrchcity,mrchname,cnum,store_name
0,2019-09-10 00:00:00,1186.00,5411,MOSCOW,H1DE7-QV5,EEJBRN,Лента
1,2019-09-10 00:00:00,434.00,5411,MOSCOW,7T 720L37T3LK7,MQRJRJ,Азбука Вкуса
2,2019-09-10 00:00:00,643.00,5411,MOSCOW,H1DE7-QV5,ELTM70,Лента
3,2019-09-10 00:00:00,370.00,5411,MOSCOW,T1H434O,EEJU6U,Великий
4,2019-09-10 00:00:00,44.00,5411,MOSCOW,H1DE7-BSM,EEIRIG,Лента
...,...,...,...,...,...,...,...
5712003,2019-12-19 00:00:00,39.00,5411,MOSCOW,7LIA7D I4EO CXK3T7,E4AUZ2,Ашан
5712004,2019-12-19 00:00:00,1025.86,5411,MOSCOW,7LIA7D VQF 3LDEK1TX,BGSCCF,Ашан Кунцево
5712005,2019-12-19 00:00:00,3573.66,5411,MOSCOW,7LIA7D VQF 3LDEK1TX,ELDCK9,Ашан Кунцево
5712006,2019-12-19 00:00:00,110.35,5411,MOSCOW,7LIA7D VQF 3LDEK1TX,ASSXCC,Ашан Кунцево


### Удаляем выбросы и выбираем магазин-клиент в качестве таргета

In [21]:
merge_data.amount.describe()

count    5.712008e+06
mean     8.027910e+02
std      1.589130e+03
min      1.000000e-02
25%      1.680000e+02
50%      3.817600e+02
75%      8.746000e+02
max      1.081005e+06
Name: amount, dtype: float64

In [23]:
merge_data = merge_data.loc[merge_data['amount'] < 25000.00]
merge_data = merge_data.loc[merge_data['amount'] > 20.00]
merge_data.shape

(5664918, 7)

In [26]:
merge_data.loc[merge_data['store_name'] == 'ашан рязанка', 'store_name'] = 'ашан'
merge_data.loc[merge_data['store_name'] == 'ашан рублевка', 'store_name'] = 'ашан'
merge_data.loc[merge_data['store_name'] == 'ашан сокольники', 'store_name'] = 'ашан'
merge_data.loc[merge_data['store_name'] == 'ашан кунцево', 'store_name'] = 'ашан'
merge_data.loc[merge_data['store_name'] == 'ашан рублевка', 'store_name'] = 'ашан'
merge_data.loc[merge_data['store_name'] == 'ашан гагаринский', 'store_name'] = 'ашан'
merge_data.loc[merge_data['store_name'] == 'ашан сокольники', 'store_name'] = 'ашан'
merge_data.loc[merge_data['store_name'] == 'ашан кунцево', 'store_name'] = 'ашан'
merge_data.loc[merge_data['store_name'] == 'ашан гагаринский ', 'store_name'] = 'ашан'

merge_data.loc[merge_data['store_name'] == 'глобус красногорск', 'store_name'] = 'глобус'
merge_data.loc[merge_data['store_name'] == 'метро стор 1017', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1356', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1014', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1073', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1019', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1061', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1077', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1048', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1050', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1318', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1322', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1011', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1049', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1012', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1018', 'store_name'] = 'metro cash&carry'
merge_data.loc[merge_data['store_name'] == 'метро стор 1049', 'store_name'] = 'metro cash&carry'

merge_data.loc[merge_data['store_name'] == 'ав экспресс меню', 'store_name'] = 'азбука вкуса'
merge_data.loc[merge_data['store_name'] == 'азбука вкуса daily', 'store_name'] = 'азбука вкуса'

In [30]:
array1 = merge_data.store_name.value_counts().index.tolist()
array2 = merge_data.store_name.value_counts().tolist()
array3 = []
for i, j in zip(array1, array2):
    couple = i + ' ' + str(j)
    array3.append(couple)
array3

['Пятерочка 1173115',
 'Перекресток 977340',
 'Вкусвилл 712524',
 'Азбука Вкуса 457181',
 'Магнит 344221',
 'Магнолия 237137',
 'Дикси 212835',
 'Мясновъ 161927',
 'SPAR 139304',
 'Мираторг 124378',
 'Ашан 124240',
 'Fix Price 82901',
 'Окей 70796',
 'Metro Cash&Carry 56695',
 'Лента 48762',
 'Глобус Красногорск 43635',
 'Глобус 37021',
 'Виктория 35695',
 'Атак 33401',
 'Ярче! 30747',
 'Ашан Гагаринский 30736',
 'Продукты 26506',
 'Ашан Сокольники 25672',
 'Ашан Кунцево 20172',
 'Да! 19386',
 'Карусель 18650',
 'Добрынинский 18518',
 'Авоська 16178',
 'Ашан Рязанка 15417',
 'Ашан Рублевка 14756',
 'Азбука вкуса 12910',
 'Продуктовый Магазин 11022',
 'Eurospar 10505',
 'Гастроном 10366',
 'Метро Стор 1017 10246',
 'Густо 10238',
 'Алые Паруса 9271',
 'Метро Стор 1356 9213',
 'Супермаркет Сити 9133',
 'IKEA 8958',
 'Вивомаркет 8945',
 'Верный 8776',
 'Метро Стор 1014 8765',
 'Адмирал 8048',
 'Фасоль 7991',
 'Метро Стор 1073 7529',
 'Метро Стор 1019 7314',
 'Фреш Маркет 7054',
 'Утконос 